In [ ]:
#Perform classficiation on Fashion MNIST with the imported model from MNIST digits

In [15]:
import torch
import torch.nn as nn
from keras.datasets import mnist
from torch import optim
from torch.utils.data import Dataset, DataLoader


class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Conv2d(1, 64, kernel_size = 4), 
        nn.ReLU(), 
        nn.MaxPool2d((2, 2), stride = 2), 
        nn.Conv2d(64, 128, kernel_size = 3), 
        nn.ReLU(), 
        nn.MaxPool2d((2, 2), stride = 2), 
        nn.Conv2d(128, 64, kernel_size = 3), 
        nn.ReLU(), 
        nn.MaxPool2d((2, 2), stride = 2))
        self.classification_head = nn.Sequential(nn.Linear(64, 20, bias = True),
        nn.ReLU(), 
        nn.Linear(20, 10, bias = True))

    def forward(self, x):
        features = self.net(x)
        return self.classification_head(features.view(1, -1))
        
class customdataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
        self.y = self.y.type(torch.LongTensor)

    def size(self):
        return len(self.x)

    def get_item(self, index):
        return [self.x[index], self.y[index]]


In [16]:
m2 = CNN()
m2 = torch.load("m_modified.pt")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
m2.to(device)
print(device)
print("State dictionary:")
for param in m2.state_dict().keys():
    print(param, "\t", m2.state_dict()[param].size())
    

cuda
State dictionary:
net.0.weight 	 torch.Size([64, 1, 4, 4])
net.0.bias 	 torch.Size([64])
net.3.weight 	 torch.Size([128, 64, 3, 3])
net.3.bias 	 torch.Size([128])
net.6.weight 	 torch.Size([64, 128, 3, 3])
net.6.bias 	 torch.Size([64])
classification_head.0.weight 	 torch.Size([20, 64])
classification_head.0.bias 	 torch.Size([20])
classification_head.2.weight 	 torch.Size([10, 20])
classification_head.2.bias 	 torch.Size([10])


In [40]:
m2.eval()
correct = 0
total = 0 
(train_x, train_y), (test_x, test_y) = mnist.load_data()
data = customdataset(test_x, test_y)
batch_size = 1
test_loader = DataLoader(list(zip(data.x, data.y)), batch_size, shuffle = True)
for i, vdata in enumerate(test_loader):
    tinputs, tlabels = vdata
    tinputs = tinputs.to(device)
    tlabels = tlabels.to(device)
    toutputs = m2(tinputs)
    _, predicted = torch.max(toutputs, 1)
    total += tlabels.size(0)
    correct += (predicted == tlabels).sum()
accuracy = 100*correct/total
print("The overall accuracy is ", accuracy)

The overall accuracy is  tensor(96.6300, device='cuda:0')


In [50]:
import torch 
from torchvision import datasets, transforms 

transform = transforms.Compose([transforms.ToTensor()])
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data', download = True, train= True, transform = transform)


trainloader = torch.utils.data.DataLoader(trainset, batch_size=1)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(m2.parameters(), lr=0.01)
losses = []

for epoch in range(2):
    m2.train(True)
    loss_t = 0
    for i, data in enumerate(trainloader):
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda() 
        optimizer.zero_grad()
        outputs = m2.forward(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        loss_t += loss.item()
        if i%10000 == 0:
            print("Data entry:", i)

Data entry: 0
Data entry: 10000
Data entry: 20000
Data entry: 30000
Data entry: 40000
Data entry: 50000
Data entry: 0
Data entry: 10000
Data entry: 20000
Data entry: 30000
Data entry: 40000
Data entry: 50000


In [51]:
#test accuracy 
m2.eval()
correct = 0
total = 0 

batch_size = 1
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data', download = True, train= False, transform = transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=1)

for i, vdata in enumerate(test_loader):
    tinputs, tlabels = vdata
    tinputs = tinputs.to(device)
    tlabels = tlabels.to(device)
    toutputs = m2(tinputs)
    _, predicted = torch.max(toutputs, 1)
    total += tlabels.size(0)
    correct += (predicted == tlabels).sum()
accuracy = 100*correct/total
print("The overall accuracy is ", accuracy)

The overall accuracy is  tensor(84.4300, device='cuda:0')


In [4]:
#2. Learn the AlexNet architecture and apply transfer learning to perform the classification 
#task. Using the pre-trained AlexNet, classify images from the cats_and_dogs_filtered 
#dataset downloaded from the below link. Finetune the classifier given in Alexnet
import PIL.Image
import torch 
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torchvision import transforms
import glob
from torchvision.datasets import ImageFolder
from torchvision.models import AlexNet_Weights 

model = torch.hub.load('pytorch/vision:v0.10.0', model='alexnet',weights=AlexNet_Weights.DEFAULT)
batch_size = 4
loss_fn = torch.nn.CrossEntropyLoss()
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 2)



Using cache found in /home/student/.cache/torch/hub/pytorch_vision_v0.10.0


In [33]:
import os
directory = 'cats_and_dogs_filtered/validation/'
classes = os.listdir(directory)
cats = os.listdir(directory + "/cats")
print(classes)
print('No. of training examples for cats:', len(cats))
dogs = os.listdir(directory + "/dogs")
print('No. of training examples for dogs:', len(dogs))

['dogs', 'cats']
No. of training examples for cats: 500
No. of training examples for dogs: 500


In [14]:
transform = transforms.Compose([transforms.Resize((100,100)),transforms.ToTensor()])
dataset = ImageFolder(directory, transform=transform)
# print example
# img, label = dataset[0]
# print(img)
# print(label)
loader = DataLoader(dataset, batch_size)
device = 'cuda'
model.to(device)
model.eval()
correct = 0
total = 0 
for i, vdata in enumerate(loader):
    tinputs, tlabels = vdata
    tinputs = tinputs.to(device)
    tlabels = tlabels.to(device)
    toutputs = model(tinputs)
    _, predicted = torch.max(toutputs, 1)
    total += tlabels.size(0)
    correct += (predicted == tlabels).sum()
accuracy = 100*correct/total
print("The overall accuracy is ", accuracy)

The overall accuracy is  tensor(45.0000, device='cuda:0')


In [ ]:
#3.Implement check points in PyTorch by saving model state_dict, optimizer state_dict, epochs
#and loss during training so that the training can be resumed at a later point. Also, illustrate
#the use of check point to save the best found parameters during training.

In [35]:
#training classifier section
directory = 'cats_and_dogs_filtered/train/'
transform = transforms.Compose([transforms.Resize((100,100)),transforms.ToTensor()])
dataset = ImageFolder(directory, transform=transform)
# print(len(dataset))
train_loader = DataLoader(dataset, batch_size)
device = 'cuda'
optimizer = optim.SGD(m2.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()
model.train()

print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

Model's state_dict:
features.0.weight 	 torch.Size([64, 3, 11, 11])
features.0.bias 	 torch.Size([64])
features.3.weight 	 torch.Size([192, 64, 5, 5])
features.3.bias 	 torch.Size([192])
features.6.weight 	 torch.Size([384, 192, 3, 3])
features.6.bias 	 torch.Size([384])
features.8.weight 	 torch.Size([256, 384, 3, 3])
features.8.bias 	 torch.Size([256])
features.10.weight 	 torch.Size([256, 256, 3, 3])
features.10.bias 	 torch.Size([256])
classifier.1.weight 	 torch.Size([4096, 9216])
classifier.1.bias 	 torch.Size([4096])
classifier.4.weight 	 torch.Size([4096, 4096])
classifier.4.bias 	 torch.Size([4096])
classifier.6.weight 	 torch.Size([2, 4096])
classifier.6.bias 	 torch.Size([2])
Optimizer's state_dict:
state 	 {}
param_groups 	 [{'lr': 0.01, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'maximize': False, 'foreach': None, 'differentiable': False, 'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}]


In [37]:
for feature in model.features:
    feature.trainable = False
model.compile()
model.to('cpu')

for epoch in range(10):
    running_loss = 0
    for i, vdata in enumerate(train_loader):
        tinputs, tlabels = vdata
        toutputs = model(tinputs)
        loss = loss_fn(toutputs, tlabels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print("[ " + str(epoch) + " ] loss:" + str(running_loss))       


[ 0 ] loss:470.59267880767584
[ 1 ] loss:471.9527007192373
[ 2 ] loss:469.7962705641985
[ 3 ] loss:473.22094079852104
[ 4 ] loss:470.1583635658026
[ 5 ] loss:469.27484180033207
[ 6 ] loss:468.8068710491061
[ 7 ] loss:477.7873417586088
[ 8 ] loss:469.6270612254739
[ 9 ] loss:467.2623813673854


In [41]:
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': running_loss,
            }, 'm1.pt')
checkpoint = torch.load('m1.pt')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']